In [11]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]

while(True):
    # ユーザーからの質問を受付
    question = input("メッセージを入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break
    display(f"質問:{question}")

    # メッセージにユーザーからの質問を追加
    messages.append({"role": "user", "content": question.strip()})
    # やりとりが8を超えたら古いメッセージから削除
    if len(messages) > 8:
        del_message = messages.pop(0)

    # 言語モデルに質問
    response_message = process_response(question, tools)

    # メッセージに言語モデルからの回答を追加
    print(response_message, flush=True)
    messages.append({"role": "assistant", "content": response_message})

print("\n---ご利用ありがとうございました！---")

------define_tools(ツール定義)------


'質問:埼玉駅のイベントについて、最近1ヶ月以内の検索結果を教えてください'

最近1ヶ月以内の埼玉駅周辺でのイベント情報は以下の通りです：

1. **埼玉県のお祭り - ジモティー**
   - 日時: 1月18日(日)
   - 場所: キッチンパートナーBIJOGI BASE
   - 内容: 中華料理、カレー、チーズバーガーなどの食材の販売。
   - 詳細: [ジモティーイベント](https://jmty.jp/saitama/eve-fes?srsltid=AfmBOorhjMUqh3EzsXp53cQHbDicG7bJor35F5099feEtYI-sOW8R0Md)

2. **記念すべき第一回 - インスタイベント**
   - 日時: 2月21日(土)
   - 場所: 埼玉・浦和駅東口 (PARCO前)
   - 内容: コスプレイベントが開催されます。
   - 詳細: [インスタグラムリンク](https://www.instagram.com/p/DTsl3nmklh9/)

3. **JR大宮駅での埼玉食品の魔力再発見「もぐもぐ埼玉」**
   - 内容: 埼玉県の特産品や料理を紹介するイベント。
   - 詳細: [記事](https://japan.storm.mg/articles/1101057)

4. **エルミこうのすのイベント**
   - 日時: 2月18日(土)〜2月19日(日)
   - 内容: 様々なショップが出展するマーケットイベント。
   - 詳細: [エルミ公式サイト](https://www.elumikonosu.com/event/)

これらのイベントは埼玉駅からアクセスしやすい場所で行われるため、興味がある方はぜひ参加してみてください。

---ご利用ありがとうございました！---


In [9]:
# 言語モデルへの質問を行う関数
def ask_question(question, tools):
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[{"role": "user", "content": question}],
        tools=tools,
        tool_choice="auto",
    )
    return response

In [10]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response(question, tools)
print(response_message)

------define_tools(ツール定義)------
最近1ヶ月以内の東京駅でのイベント開催情報は以下の通りです。

1. **東京駅のイベント（駅探）**
   - 開催期間: 詳細な期間は記載されていませんが、定期的にイベントが行われています。
   - 内容: 日本の伝統料理やアートイベントなど、多様なイベントが行われます。
   - [イベント情報はこちら](https://ekitan.com/event/station-2590)

2. **関東のイベント（WalkerPlus）**
   - 開催期間: 2025年から2026年の範囲でイベントが計画されています。
   - 内容: 東京駅周辺での文化イベントや博覧会、シーズナルイベントなど。
   - [イベント情報はこちら](https://www.walkerplus.com/event_list/ar0300/)

3. **今日行ける東京駅周辺のイベント（Enjoy Tokyo）**
   - 開催期間: 2026年までの様々なイベント情報が掲載されています。
   - 内容: 期間限定のアート展やフードイベントなど。
   - [イベント確認はこちら](https://www.enjoytokyo.jp/event/list/area1306/its04/)

4. **MarkeZine Day 2026（マーケティングイベント）**
   - 開催日: 2026年3月1日、2日
   - 内容: マーケティングに関連するトピックや技術に関するイベント。
   - [詳細情報](https://markezine.jp/article/detail/50440)

これらの情報は、東京駅周辺での最近のイベントに関するものですので、興味のある方はぜひリンクを参考にしてください。


In [7]:
# ユーザーからの質問を処理する関数
def process_response(question, tools):
    response = ask_question(question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [6]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=[
            {"role": "user", "content": question},
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [5]:
# ツール定義
def define_tools():
    print("------define_tools(ツール定義)------")
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "最近一ヵ月のイベント開催予定などネット検索が必要な場合に、質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [4]:
# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://ekitan.com/event/station-2590',
   'title': '東京駅周辺のイベント - 駅探',
   'content': '## 東京駅のイベント一覧. ### 条件指定. * #### 日本の桜名所100選\u3000赤城南面千本桜！菜の花・桜・花桃の中を駆け抜けるトロッコわたらせ渓谷号. + 二重橋前駅／東京駅／大手町駅(東京)駅. * #### 平日昼間\u3000初心者向け 山手線1周ラン 約10～42キロ（キロ約7～8分）. + 東京駅／二重橋前駅／京橋駅(東京)駅. + 二重橋前駅／東京駅／大手町駅(東京)駅. * #### 山手線1周ウォーク 約10～42キロ（時速3～4キロ） 11月. + 東京駅／二重橋前駅／京橋駅(東京)駅. * #### 山手線1周ウォーク 約10～42キロ（時速3～4キロ） 5月. + 東京駅／二重橋前駅／京橋駅(東京)駅. * #### 初心者向け 山手線1周ラン 約10～42キロ（キロ約7～8分） 4月. + 東京駅／二重橋前駅／京橋駅(東京)駅. * #### 初心者向け 山手線1周ラン 約10～42キロ（キロ約7～8分） 4月. + 東京駅／二重橋前駅／京橋駅(東京)駅. * #### 初心者向け ゲートブリッジラン 約15～60キロ（キロ約7～8分）. + 東京駅／二重橋前駅／京橋駅(東京)駅. ### 近くの駅のイベント一覧. ### 東京駅沿線のイベント一覧. ### 東京駅の周辺情報. ### 東京駅の路線図・停車駅. ### 東京駅の運行情報. ### 東京駅発の人気の区間(乗換案内). ### 関連サービス. ### **おすすめ記事**.',
   'score': 0.8170061,
   'raw_content': None},
  {'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/',
   'title': '東京駅(東京都)周辺のイベント - ウォーカープラス',
   'content': '開催中 2025年12月20日(土)～2026年2月22日(日). 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座

In [3]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

In [2]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"
